In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [3]:
#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class Bollinger_Breakout(bt.Indicator):
    lines = ('breakout',)
    
    def __init__(self):
        self.lines.breakout = self.data > bt.indicators.BBands(self.data, period=20).lines.top

In [4]:
class Bollinger_Breakdown(bt.Indicator):
    lines = ('breakdown',)
    
    def __init__(self):
        self.lines.breakdown = self.data < bt.indicators.BBands(self.data, period=20).lines.bot

In [5]:
class Bollinger_Upperbelt(bt.Indicator):
    lines = ('upperbelt',)
    
    def __init__(self):
#         if (self.data > bt.indicators.BBands(self.data, period=20).lines.mid and self.data < bt.indicators.BBands(self.data, period=20).lines.top):
#             self.lines.upperbelt = True
#         else:
#             self.lines.upperbelt = False
#         print(type(self.lines.upperbelt))
#         print(type(self.data))
#         self.lines.upperbelt = self.data
#         if (self.lines.upperbelt > 20):
#             self.lines.upperbelt = True
#         else:
#             self.lines.upperbelt = False

        self.lines.upperbelt = self.data > bt.indicators.BBands(self.data, period=20).lines.mid

In [6]:
class Bollinger_Lowerbelt(bt.Indicator):
    lines = ('lowerbelt',)
    
    def __init__(self):

        self.lines.lowerbelt = self.data < bt.indicators.BBands(self.data, period=20).lines.mid

In [11]:
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 4),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
#         self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
#         self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
#         self.dsma = self.sma_short - self.sma_long
#         self.mom = bt.Cmp(self.sma_short, self.sma_long)


#         self.bollinger_upperbelt_1 = self.stock_px > bt.indicators.BBands(self.data, period=20).lines.mid
#         self.bollinger_upperbelt_2 = self.stock_px < bt.indicators.BBands(self.data, period=20).lines.top
        
#         self.bollinger_lowerbelt_1 = self.stock_px > bt.indicators.BBands(self.data, period=20).lines.bot
#         self.bollinger_lowerbelt_2 = self.stock_px < bt.indicators.BBands(self.data, period=20).lines.mid
 

        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.bollinger_breakout = Bollinger_Breakout(self.stock_px)
        self.bollinger_breakdown = Bollinger_Breakdown(self.stock_px)
#        self.bollinger_upperbelt = Bollinger_Upperbelt(self.stock_px)
#         self.bollinger_lowerbelt = Bollinger_Lowerbelt(self.stock_px)
        
        #self.bollinger_opening = Bollinger_Opening(self.stock_px)
            
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
#        self.rsi_oversold = bt.indicators.RSI(self.stock_px) < 30
#        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70

#         self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
#         self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size

#         # If not in the trade
#         if (self.flag == 0):
#             if ((self.bollinger_breakout[0] > 0 or self.bollinger_breakdown[0] > 0) and self.days_in_trade < self.p.max_days_in_trade):
#                 print("Breakout indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
                
#             '''
#             if (self.bollinger_upperbelt[-3] > 0 and self.bollinger_upperbelt[-2] and self.bollinger_upperbelt[-1] > 0  and self.bollinger_breakdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("111Breakdown indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
            
#             if (self.bollinger_lowerbelt[-3] > 0 and self.bollinger_lowerbelt[-2] > 0 and self.bollinger_lowerbelt[-1] > 0 and self.bollinger_breakout[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("111Breakout indicator triggered...entering position")
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = -1
            
#             if (self.bollinger_breakdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("Breakdown indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = -1
#             '''
            
#         # If in the long position
#         elif (self.flag == 1):
#             # Close the position if the SELL signal triggered or reached the max days to hold
#             if (self.bollinger_breakdown[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                 #self.bollinger_upperbelt[3] > 0 and self.bollinger_upperbelt[2] and self.bollinger_upperbelt[1] > 0 Holding for three days?
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.bollinger_breakout[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 print("In position %d days" % self.days_in_trade)
        
#         # If in the short position
#         elif (self.flag == -1):
#             # Close the position if the BUY signal triggered or reached the max days to hold
#             if (self.bollinger_breakout[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 print("Closing the position")
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.bollinger_breakdown[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 print("In position %d days" % self.days_in_trade)
                
#         print("Flag = %d" % self.flag)
#         print("In position %d days" % self.days_in_trade)
            
        
        #Check if we are currently in a trade
        
        
 #===============================================================================================       
#         if (self.bollinger_breakdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):             
#             self.buy(exectype=bt.Order.Close)
#             self.days_in_trade = 1
#             self.flag = 1
            

#         self.days_in_trade += 1
        
#         if (self.days_in_trade == self.p.max_days_in_trade):
#             self.close(exectype=bt.Order.Close)
#             self.days_in_trade = 0
#===============================================================================================


        if(not size):
#             If not has our indicator triggered?
            if self.bollinger_breakdown[0] > 0:
#                 print("Oversold indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                
#             if (self.bollinger_breakdown[0] > 0 and self.bollinger_lowerbelt_1[-3] > 0 and self.bollinger_lowerbelt_1[-2] >0 and self.bollinger_lowerbelt_1[-1] > 0):
# #                 print("Overbought indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close,size=1)
#                 self.days_in_trade = 1
                               
        else:
            self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            
            if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0

In [14]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/YHOO.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Add Analyzers
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown,_name="DrawDown")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe',timeframe=7)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TA')
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='HV-YHOO-BBands-Breakdown.csv',csv=True)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())  
    print('Sharpe Ratio:', thestrat.analyzers.mySharpe.get_analysis())
    print('Max Draw Down:',thestrat.analyzers.DrawDown.get_analysis(),"\n")
    #print('Trade Stats:',thestrat.analyzers.TA.get_analysis())
    for k,v in thestrat.analyzers.TA.get_analysis().items():
        print(k,":",v,"\n")

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100091.70
Sharpe Ratio: OrderedDict([('sharperatio', -23.770262596458664)])
Max Draw Down: OrderedDict([('maxdrawdown', 0.009287260258217425), ('maxdrawdownperiod', 415)]) 

total : AutoOrderedDict([('total', 46), ('open', 0), ('closed', 46)]) 

streak : AutoOrderedDict([('won', AutoOrderedDict([('current', 1), ('longest', 5)])), ('lost', AutoOrderedDict([('current', 0), ('longest', 3)]))]) 

pnl : AutoOrderedDict([('gross', AutoOrderedDict([('total', 91.7), ('average', 1.9934782608695654)])), ('net', AutoOrderedDict([('total', 91.7), ('average', 1.9934782608695654)]))]) 

won : AutoOrderedDict([('total', 34), ('pnl', AutoOrderedDict([('total', 125.90999999999998), ('average', 3.7032352941176465), ('max', 12.41)]))]) 

lost : AutoOrderedDict([('total', 12), ('pnl', AutoOrderedDict([('total', -34.209999999999994), ('average', -2.8508333333333327), ('max', -7.590000000000003)]))]) 

long : AutoOrderedDict([('total', 46), ('pnl', 